In [160]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [161]:
df= pd.read_csv('/DFORE353.csv')

In [162]:
df.isnull().sum()

machine_type            0
controller              0
client_account_id       0
cycle_type             41
vends                   0
Spend                2646
UnitSpend            2646
dtype: int64

In [163]:
df.dropna(inplace=True)

In [164]:
df['machine_type']= df.machine_type.str.upper()

In [165]:
df1=df.loc[df.machine_type=='WASHER'].loc[~df.cycle_type.isin(['[""]', '["\\u0001"]', '["\\u0004"]','[]'])]

In [166]:
df1.head()

,machine_type,controller,client_account_id,cycle_type,vends,Spend,UnitSpend
32371,WASHER,ACA,4420522,"[""cold""]",40,7160.0,179.0
32372,WASHER,Other,820249,"[""cold""]",7,1050.0,150.0
32373,WASHER,Other,2720827,"[""cold""]",89,13950.0,156.0
32374,WASHER,Other,7312119,"[""cold""]",85,10675.0,125.0
32375,WASHER,Other,4004189,"[""cold""]",1,150.0,150.0


In [167]:
df1.groupby(['cycle_type', 'controller']).agg({'client_account_id': "nunique", "UnitSpend":"mean"}).reset_index().pivot(index= 'cycle_type', columns= 'controller').sort_index()

client_account_id  ...   UnitSpend
controller                                   ACA  ...       Other
cycle_type                                        ...            
["cold"]                                     1.0  ...  168.367347
["delicates"]                                3.0  ...  173.010256
["heavy","delicates","cold"]               298.0  ...  172.056338
["heavy","delicates","hot"]                212.0  ...  170.899254
["heavy","delicates","warm"]               201.0  ...  166.567347
["heavy","normal","cold"]                 1564.0  ...  172.852995
["heavy","normal","hot"]                  1745.0  ...  175.621172
["heavy","normal","warm"]                 1630.0  ...  174.403137
["heavy","perm-press","cold"]              482.0  ...  169.524051
["heavy","perm-press","hot"]               696.0  ...  164.650728
["heavy","perm-press","warm"]              741.0  ...  167.832323
["heavy"]                                    2.0  ...  190.460745
["high-temp"]                                3.0  ...   85.195238
["light","cold"]                             3.0  ...  150.000000
["light","delicates","cold"]               455.0  ...  152.946850
["light","delicates","hot"]                100.0  ...  154.915033
["light","delicates","warm"]               201.0  ...  153.751908
["light","normal","cold"]                 2559.0  ...  161.888679
["light","normal","hot"]                  1363.0  ...  152.833478
["light","normal","warm"]                 1472.0  ...  153.804962
["light","normal"]                           7.0  ...  132.400000
["light","perm-press","cold"]              508.0  ...  152.595062
["light","perm-press","hot"]               458.0  ...  153.361217
["light","perm-press","warm"]             1262.0  ...  156.236534
["light","perm-press"]                       3.0  ...         NaN
["low-temp"]                                 1.0  ...   81.507599
["med-temp"]                                12.0  ...   86.063830
["medium","delicates","cold"]              270.0  ...  162.048443
["medium","delicates","hot"]                79.0  ...  166.977528
["medium","delicates","warm"]              177.0  ...  159.936585
["medium","normal","cold"]                1359.0  ...  160.600589
["medium","normal","hot"]                 1136.0  ...  160.623773
["medium","normal","warm"]                1503.0  ...  161.000971
["medium","normal"]                          1.0  ...         NaN
["medium","perm-press","cold"]             465.0  ...  157.306733
["medium","perm-press","hot"]              350.0  ...  156.350365
["medium","perm-press","warm"]             752.0  ...  154.162162
["normal"]                                   3.0  ...  174.177437
["perm-press"]                               4.0  ...  171.806487
["small","delicates","cold"]               191.0  ...  163.825688
["small","delicates","hot"]                  8.0  ...  149.470588
["small","delicates","warm"]                26.0  ...  151.184211
["small","normal","cold"]                  197.0  ...  144.979508
["small","normal","hot"]                   171.0  ...  142.596330
["small","normal","warm"]                  175.0  ...  143.384615
["small","perm-press","cold"]               39.0  ...  157.947368
["small","perm-press","hot"]                35.0  ...  150.500000
["small","perm-press","warm"]               72.0  ...  149.959184
["warm"]                                     2.0  ...  167.826087

[49 rows x 4 columns]

In [168]:
df1.UnitSpend.describe(percentiles=[.02,.4,.6,.8])

count    56015.000000
mean       172.361635
std         49.277590
min          3.000000
2%          58.000000
40%        169.000000
50%        175.000000
60%        175.000000
80%        200.000000
max       2250.000000
Name: UnitSpend, dtype: float64

In [169]:
p5= df1.UnitSpend.describe(percentiles=[.05,.25,.50,.75,.95]).loc['5%']
p95= df1.UnitSpend.describe(percentiles=[.05,.25,.50,.75,.95]).loc['95%']
df1["UnitSpend"] = np.where(df1["UnitSpend"] <p5, p5,df1['UnitSpend'])
df1["UnitSpend"] = np.where(df1["UnitSpend"] >p95, p95,df1['UnitSpend'])

In [170]:
df1['Spndbkt']= pd.qcut(df1.UnitSpend, 4, labels=['low', 'medium','high', 'Uber'])

In [171]:
pd.qcut(df1.UnitSpend, 4).value_counts()

(92.999, 150.0]    19211
(150.0, 175.0]     16233
(175.0, 200.0]     11032
(200.0, 250.0]      9539
Name: UnitSpend, dtype: int64

In [172]:
df1.groupby(['cycle_type', 'controller', 'Spndbkt']).agg({'client_account_id':'nunique', 'vends': 'sum'}).reset_index().pivot(index=['cycle_type', 'controller'], columns=['Spndbkt']).reset_index().to_csv('DFORE353.csv')

In [173]:
df1['CycleTypeBkt']= 'Others'

In [174]:
df1.head()

,machine_type,controller,client_account_id,cycle_type,vends,Spend,UnitSpend,Spndbkt,CycleTypeBkt
32371,WASHER,ACA,4420522,"[""cold""]",40,7160.0,179.0,high,Others
32372,WASHER,Other,820249,"[""cold""]",7,1050.0,150.0,low,Others
32373,WASHER,Other,2720827,"[""cold""]",89,13950.0,156.0,medium,Others
32374,WASHER,Other,7312119,"[""cold""]",85,10675.0,125.0,low,Others
32375,WASHER,Other,4004189,"[""cold""]",1,150.0,150.0,low,Others


In [175]:
df1.loc[df1.cycle_type.isin(['["heavy","delicates","cold"]',
'["heavy","delicates","hot"]',
'["heavy","delicates","warm"]',
'["heavy","normal","cold"]',
'["heavy","normal","hot"]',
'["heavy","normal","warm"]',
'["heavy","perm-press","cold"]',
'["heavy","perm-press","hot"]',
'["heavy","perm-press","warm"]']), 'CycleTypeBkt']= 'Premium'

In [176]:
df1.loc[df1.cycle_type=='["light","normal","cold"]', 'CycleTypeBkt']= 'Normal'

In [177]:
df1.CycleTypeBkt.value_counts()

Others     38729
Premium    13137
Normal      4149
Name: CycleTypeBkt, dtype: int64

In [178]:
df2= df1.loc[df1.controller=='ACA'].groupby(['client_account_id', 'CycleTypeBkt']).agg({'vends': 'sum', 'Spend': 'sum'}).reset_index()

In [179]:
df2['UnitSpend']= df2.Spend/df2.vends

In [182]:
df3=df2.pivot(index='client_account_id', columns='CycleTypeBkt', values='UnitSpend')

In [187]:
df3['Premium-Normal/ Normal']= (df3['Premium']- df3['Normal'])/df3['Normal']

In [189]:
df4= df3.dropna()

In [203]:
df4['PriceComparison']= pd.cut(df4['Premium-Normal/ Normal'], [-99,0,.1,.2,.3, 99], labels=['Premium is low!', 'About 10% Higer', 'Between 10-20% Higher', 'Between 20-30% Higher', 'More than 30% Higher'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [214]:
df4.reset_index(inplace=True)

In [215]:
df.columns

Index(['machine_type', 'controller', 'client_account_id', 'cycle_type',
       'vends', 'Spend', 'UnitSpend'],
      dtype='object')

In [217]:
df4.groupby('PriceComparison').agg({'client_account_id':'nunique','Normal':'mean', 'Premium': 'mean'})

CycleTypeBkt,client_account_id,Normal,Premium
PriceComparison,,,
Premium is low!,475,173.766800,172.195151
About 10% Higer,248,172.467056,177.476428
Between 10-20% Higher,111,190.872658,220.840337
Between 20-30% Higher,510,184.818381,233.283252
More than 30% Higher,315,145.181501,196.842398


In [230]:
{
    'All AS400:' :len(df1.client_account_id.unique()),
    'ACA AS400:' :len(df2.client_account_id.unique()),
    'Premium+Normal Settings AS400': len(df4.client_account_id.unique())
 }

{'ACA AS400:': 3674, 'All AS400:': 8474, 'Premium+Normal Settings AS400': 1659}

## Parse

In [240]:
df.loc[df.controller=='ACA'].loc[df.machine_type=='WASHER'].groupby('cycle_type').agg({'vends':'sum', 'Spend':'sum'}).sort_values('Spend', ascending= False).reset_index()

,cycle_type,vends,Spend
0,"[""light"",""normal"",""cold""]",718520,122819643.0
1,"[""light"",""perm-press"",""warm""]",271650,47504277.0
2,"[""heavy"",""normal"",""hot""]",119130,22837940.0
3,"[""medium"",""normal"",""warm""]",116706,20811129.0
4,"[""heavy"",""normal"",""warm""]",103429,19367059.0
5,"[""light"",""normal"",""warm""]",111554,19176227.0
6,"[""heavy"",""normal"",""cold""]",91902,17301899.0
7,[],74524,13295475.0
8,"[""medium"",""normal"",""cold""]",69560,12548180.0
9,"[""light"",""normal"",""hot""]",63304,11027085.0
